In [1]:
from google.colab import drive
import seaborn as sns
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor  
import random

In [2]:
drive.mount('/content/drive')
train_df=pd.read_csv('/content/drive/My Drive/Participants_Data_TGIH/Train.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
test_df=pd.read_csv('/content/drive/My Drive/Participants_Data_TGIH/Test.csv')

In [4]:
train_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,6141,1583,144,3,2011-05-06 16:54:00,3.75,14056.0,35
1,6349,1300,3682,6,2011-05-11 07:35:00,1.95,13098.0,35
2,16783,2178,1939,4,2011-11-20 13:20:00,5.95,15044.0,35
3,16971,2115,2983,1,2011-11-22 12:07:00,0.83,15525.0,35
4,6080,1210,2886,12,2011-05-06 09:00:00,1.65,13952.0,35


In [5]:
train_df.isnull().values.any()

False

In [6]:
train_df.shape

(284780, 8)

In [7]:
train_df.duplicated().sum()

2621

In [8]:
train_df.drop_duplicates(inplace=True)
train_df['InvoiceDate']=pd.to_datetime(train_df['InvoiceDate'])
train_df['InvoiceDay']=train_df['InvoiceDate'].dt.day
train_df['InvoiceMonth']=train_df['InvoiceDate'].dt.month
train_df['InvoiceYear']=train_df['InvoiceDate'].dt.year
train_df['InvoiceHour']=train_df['InvoiceDate'].dt.hour

In [9]:
train_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceDay,InvoiceMonth,InvoiceYear,InvoiceHour
0,6141,1583,144,3,2011-05-06 16:54:00,3.75,14056.0,35,6,5,2011,16
1,6349,1300,3682,6,2011-05-11 07:35:00,1.95,13098.0,35,11,5,2011,7
2,16783,2178,1939,4,2011-11-20 13:20:00,5.95,15044.0,35,20,11,2011,13
3,16971,2115,2983,1,2011-11-22 12:07:00,0.83,15525.0,35,22,11,2011,12
4,6080,1210,2886,12,2011-05-06 09:00:00,1.65,13952.0,35,6,5,2011,9


In [10]:
train_df.describe()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,InvoiceDay,InvoiceMonth,InvoiceYear,InvoiceHour
count,282159.000000,282159.000000,282159.000000,282159.000000,282159.000000,282159.000000,282159.000000,282159.000000,282159.000000,282159.000000,282159.000000
mean,9949.557267,1573.707527,2024.261955,12.109885,3.460501,15283.444366,32.805964,15.032244,7.604893,2010.934246,12.733597
std,5547.353569,843.814154,1089.992911,296.463809,78.762448,1714.633177,6.768422,8.651768,3.414620,0.247852,2.291076
min,0.000000,0.000000,0.000000,-80995.000000,0.000000,12346.000000,0.000000,1.000000,1.000000,2010.000000,6.000000
25%,5069.000000,942.000000,1141.000000,2.000000,1.250000,13947.000000,35.000000,7.000000,5.000000,2011.000000,11.000000
50%,10294.000000,1522.000000,1987.000000,5.000000,1.950000,15147.000000,35.000000,15.000000,8.000000,2011.000000,13.000000
75%,14640.000000,2106.000000,2945.000000,12.000000,3.750000,16791.000000,35.000000,22.000000,11.000000,2011.000000,14.000000
max,22188.000000,3683.000000,3895.000000,80995.000000,38970.000000,18287.000000,36.000000,31.000000,12.000000,2011.000000,20.000000


In [11]:
train_df['WeekDay']= train_df['InvoiceDate'].dt.dayofweek
train_df['WeekDay']=train_df['InvoiceNo'].apply(lambda x : 1 if x<5 else 0)
train_df.drop('InvoiceDate', inplace=True, axis=1)
train_df.drop('InvoiceNo', inplace=True, axis=1)
train_df.drop('InvoiceHour', inplace=True, axis=1)
train_df.drop('WeekDay', inplace=True, axis=1)

In [12]:
train_df.corr(method='spearman')['UnitPrice']

StockCode       0.098888
Description    -0.028534
Quantity       -0.408045
UnitPrice       1.000000
CustomerID     -0.013895
Country        -0.022726
InvoiceDay     -0.004739
InvoiceMonth   -0.022320
InvoiceYear    -0.031509
Name: UnitPrice, dtype: float64

In [13]:
train_df.skew()

StockCode         0.271076
Description      -0.133745
Quantity          0.223878
UnitPrice       436.324717
CustomerID        0.032807
Country          -3.034005
InvoiceDay        0.117892
InvoiceMonth     -0.440870
InvoiceYear      -3.504109
dtype: float64

In [14]:
target = train_df['UnitPrice']
train_df.drop('UnitPrice', inplace=True, axis=1)

In [15]:
train_df.head()

,StockCode,Description,Quantity,CustomerID,Country,InvoiceDay,InvoiceMonth,InvoiceYear
0,1583,144,3,14056.0,35,6,5,2011
1,1300,3682,6,13098.0,35,11,5,2011
2,2178,1939,4,15044.0,35,20,11,2011
3,2115,2983,1,15525.0,35,22,11,2011
4,1210,2886,12,13952.0,35,6,5,2011


In [16]:
x = train_df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
train_df = pd.DataFrame(x_scaled)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(train_df, target, test_size=0.33, random_state=42)

In [18]:
reg = DecisionTreeRegressor(random_state=0)

In [ ]:
param_grid = {"criterion": ["mse"], 
              # "max_features": [1,2,3,4,5,6,7,8], 
              "min_samples_split": [5, 10, 15, 20, 25, 30, 35, 40],
              "max_depth": [2, 3, 4, 5, 6, 7, 8],
              "min_samples_leaf": [20, 40, 60, 80, 100],
              "max_leaf_nodes": [5, 20, 100]
              }
## Comment in order to publish in kaggle.

grid_cv = GridSearchCV(reg, param_grid, cv=5)
grid_cv.fit(X_train, y_train)
print("Tuned Logistic Regression Parameters: {}".format(grid_cv.best_params_))  
print("Best score is {}".format(grid_cv.best_score_)) 
# print(reg.score(X_test, y_test))

In [ ]:
dtc = DecisionTreeRegressor(criterion='mse', max_depth=8, max_leaf_nodes=100, min_samples_leaf=20, min_samples_split=5, random_state=0)
dtc.fit(X_train, y_train)

In [ ]:
y_predicted = dtc.predict(X_test)

In [ ]:
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test,y_predicted)))

In [ ]:
test_df['InvoiceDate']=pd.to_datetime(test_df['InvoiceDate'])
test_df['InvoiceDay']=test_df['InvoiceDate'].dt.day
test_df['InvoiceMonth']=test_df['InvoiceDate'].dt.month
test_df['InvoiceYear']=test_df['InvoiceDate'].dt.year
test_df['InvoiceHour']=test_df['InvoiceDate'].dt.hour
test_df['WeekDay']= test_df['InvoiceDate'].dt.dayofweek
test_df['WeekDay']=test_df['InvoiceNo'].apply(lambda x : 1 if x<5 else 0)
test_df.drop('InvoiceDate', inplace=True, axis=1)
test_df.drop('InvoiceNo', inplace=True, axis=1)
test_df.drop('InvoiceHour', inplace=True, axis=1)
test_df.drop('WeekDay', inplace=True, axis=1)

In [ ]:
test_df = pd.DataFrame(min_max_scaler.transform(test_df.values))

In [ ]:
out = dtc.predict(test_df)

In [ ]:
sample_df=pd.read_csv('/content/drive/My Drive/Participants_Data_TGIH/Sample Submission.csv')

In [ ]:
sample_df['UnitPrice'] = out.round(2)

In [ ]:
sample_df

In [ ]:
sample_df.to_csv('my_submission_file19.csv', index=False)